# 说明

前面“初级、进阶”所使用的预训练的大型网络进行迁移学习，其结果已经把自己搭建的小网络的“**结果**”有了“质的提升”。并且在“进阶”部分通过改进（图片只进入卷积基一次），在保证结果精度的条件下，又使得“**计算速度**”有了质的提升。—— 至此来看，解决我们这种小的简单问题已经完全够用了。

继续提出一个问题：网络的结果能否更好？ —— 可以。

方案：对原始网络进行“**微调**”。

说明：卷积部分的底层，它们所提取的特征都是最基础、最通用、最普遍的特征；卷积层越往顶层，它们所提取的特征就越抽象、越和具体的问题有关。我们使用VGG16卷积部分的权重参数，要知道它所处理的原始问题是ImageNet大型数据集的1000分类问题！—— 可以理解：它的卷积基中越顶部的卷积层，越是提取针对其特定问题的抽象的特征！当最终所面临的问题以及数据集不同时，抽象特征理论上差距比较大！—— 因此：对于卷积基中顶部的卷积层，理论上我们也应该让它们参与训练（权重参数可修改），以此来提高它们对特征问题的高级抽象特征的提取能力！

所以：我们要微调的是卷积基部分顶部那几层卷积层（和后面Dense层紧邻的那几个卷积层）。

---

微调的步骤：
- 在预训练卷积基后面添加自定义的Dense全连接层（分类层）；
- 冻结卷积基所以层的参数；
- 训练添加的Dense全连接层； —— 前3步和前面所做的完全一样，下面才是微调的主要部分
- 解冻卷积基“顶部几层”的参数；
- 联合训练所有解冻了的卷积层和Dense全连接层。

使得顶部的卷积层，能更好的适应所面对的问题。

# 数据处理部分：和前面一样

In [1]:
import keras
from keras import layers
import numpy as np
import os
import shutil    # os和shutil用来处理文件

Using TensorFlow backend.


In [2]:
# 原始数据太多了，现在专门创建一个文件夹来存储一部分要用的训练集 + 测试集
base_dir = 'E:/Python_code/keras_total/日月光华-keras课程资料/dc/try'
train_dir = os.path.join(base_dir, 'train')
train_dir_dog = os.path.join(train_dir , 'dog')
train_dir_cat = os.path.join(train_dir , 'cat')

test_dir = os.path.join(base_dir , 'test')
test_dir_dog = os.path.join(test_dir , 'dog')
test_dir_cat = os.path.join(test_dir , 'cat')

dc_dir = 'E:/Python_code/keras_total/日月光华-keras课程资料/dc/train' # 原始数据所在路径

In [3]:
# 训练集猫狗各1000张，测试集猫狗各500张。
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
    os.mkdir(train_dir)
    os.mkdir(train_dir_dog)
    os.mkdir(train_dir_cat)
    os.mkdir(test_dir)
    os.mkdir(test_dir_dog)
    os.mkdir(test_dir_cat)

    fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname) 
        d = os.path.join(train_dir_cat, fname)
        shutil.copyfile(s, d)   #  把s拷贝到d 

    fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(test_dir_cat, fname)
        shutil.copyfile(s, d)

    fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(train_dir_dog, fname)
        shutil.copyfile(s, d)

    fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(test_dir_dog, fname)
        shutil.copyfile(s, d)

In [4]:
from keras.preprocessing.image import ImageDataGenerator

# 创建图片的迭代器，并且设定它的归一化
train_datagen = ImageDataGenerator(1/255)
test_datagen = ImageDataGenerator(1/255)

In [5]:
# 正式创建图片的生成器：train
train_generator = train_datagen.flow_from_directory( train_dir,                  # 待读取文件的目录
                                                     target_size = (200,200),    # 图片的统一大小 
                                                     batch_size = 20,            # 每次读入20张
                                                     class_mode = 'binary'       # 该文件夹下分两类：因为我已经正好在该文件夹下分了两个文件夹
)

# 正式创建图片的生成器：test
test_generator = test_datagen.flow_from_directory( test_dir,                  # 待读取文件的目录
                                                    target_size = (200,200),    # 图片的统一大小 
                                                    batch_size = 20,            # 每次读入20张
                                                    class_mode = 'binary'       # 该文件夹下分两类：因为我已经正好在该文件夹下分了两个文件夹
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# 原始预训练网络搭建：和前面一样

In [7]:
conv_base = keras.applications.VGG16( weights = 'imagenet', include_top = False, input_shape = (200,200,3))  # 每张图进入都是(200,200,3)
# 先将卷积基参数冻结：
conv_base.trainable = False   

In [8]:
# 定义一个：其作用是用卷积基来处理每张图片，获得它们对应的特征提取图，然后把特征提取图存起来
batch_size = 20

# data_generator是生成器，sample_count是样本总数
def extract_features(data_generator, sample_count):
    i = 0
    features = np.zeros( shape=(sample_count, 6, 6, 512) )  # 处理后的结果记录：把每张图变成(6,6,512)的特征提取图 —— 总数(2000,6,6,512)
    labels = np.zeros( shape=(sample_count) )
    # 特征提取 + 存储部分：20张一批一起做 —— (20, 6, 6, 512)
    for inputs_batch, labels_batch in data_generator:
        features_batch = conv_base.predict(inputs_batch)    # 每一张图，经过卷积层处理后，返回其对应的特征提取后的图！ √
        features[ i * batch_size : (i + 1) * batch_size ] = features_batch  # 把特征图存入
        labels[ i * batch_size : (i + 1) * batch_size ] = labels_batch      # 把标签存入
        i += 1
        if i * batch_size >= sample_count:
            break
            
    return features, labels 

In [ ]:
# 对训练、测试数据进行特征提取：比较慢！！！s
train_features, train_labels = extract_features(train_generator, 2000)
test_features, test_labels = extract_features(test_generator, 1000)

### 模型的搭建： 

In [9]:
# 不再需要加入那个卷积基了Sequentialquentialquentialquentialquenlayersl接把它特征提取的结果所“对应的尺寸”作为输入即可：
from keras import layers
model = keras.Sequential()
model.add( layers.GlobalAveragePooling2D(input_shape=(6, 6, 512)) )  # 这里输入是特征提取后的尺寸大小。
model.add( layers.Dense(512, activation='relu') )
model.add( layers.Dropout(0.5) )
model.add( layers.Dense(1, activation='sigmoid') )

In [ ]:
model.compile( optimizer = keras.optimizers.Adam(lr=0.001),
               loss='binary_crossentropy',
               metrics=['acc']
)

In [ ]:
# 速度会快非常多！因为只用训练Dense层的参数！！
history = model.fit( train_features, train_labels, epochs=10, batch_size=50, validation_data=(test_features, test_labels))

# 对原模型微调：

说明：到此，原模型已经训练过一遍了，即Dense层已经有了很好的参数，目前整体网络已经有了不错的结果了。下面的微调就是进一步提高它的结果。

In [11]:
conv_base.trainable = True   # 把卷积基解冻

In [13]:
len(conv_base.layers)   # 查看当前卷积基有多少个层：共19层；计划调整它“顶部/最后”的3层

19

In [14]:
# 从开头到倒数第3层冻结，最后剩下的顶部几层依旧处于打开状态：
final_layers = -3
for layer in conv_base.layers[:final_layers]:
    layer.trainable = False 

In [16]:
# 注意：optimizer这里一定要使用更低的“学习速率”来下探！因为结果已经不错的前提下，学习步长一定不能大！
model.compile( optimizer = keras.optimizers.Adam(lr=0.001/10),  # 除以10，慢10倍
               loss='binary_crossentropy',
               metrics=['acc']
)

In [ ]:
initial_epochs = 10   # 前面已经训练了10轮
final_epochs = 12     # 计划再继续训练12轮
total_epochs = initial_epochs + final_epochs  # 记录/打印的训练总轮数：22次

# 速度会变慢：因为有几个卷积层要参与计算了，并且他们还要修改自己的参数
history = model.fit( 
    train_features, 
    train_labels, 
    epochs = total_epochs, initial_epoch = initial_epochs,  #  打印/记录会从11开始，到22结束
    batch_size=50, 
    validation_data=(test_features, test_labels)
)

正确率比原始的预训练网络又提高了一些！这里的训练集仍然只用了2000张！效果已经非常好。

所以：可以先用“预训练网络 + 进阶里的方法”，快速获得一个结果很好的网络与Dense层参数；然后将卷积基部分顶部层解冻，使用微调来进一步提高网络精度（速度会变慢，因为毕竟加了一点卷积层计算在里面，且这些卷积层不仅提取特征还会更新权重）。把最后的结果保存即可。


因此：预训练模型(卷积部分) + 卷积基特征提取 + 微调 —— 可以很好、很快的解决小型数据集问题。